In [2]:
import pandas as pd
import pyreadstat


In [3]:
df_sch, meta_sch = pyreadstat.read_sav('data/PISA2022_SCH_QQQ.SAV')

In [4]:
import os

with open('data/openai.api.key', 'r') as filek: 
    openai_key = filek.read()
os.environ["OPENAI_API_KEY"] =  openai_key

In [5]:
from langchain_chroma import Chroma
from langchain_core.documents import Document
from langchain_openai import OpenAIEmbeddings

def meta2docs(spss_meta):
    excluded = ['CNTRYID']
    docs = []
    for col in  spss_meta.column_names:
        #if col in spss_meta.variable_value_labels:
        if col not in excluded:
            docs.append(
                Document(
                    page_content=spss_meta.column_names_to_labels[col],
                    metadata={"year": 2022, "original_col_name": col},
                ),
            )
    return docs
    
cols = meta2docs(meta_sch)

cols_vectorstore = Chroma.from_documents(cols, OpenAIEmbeddings())

In [6]:
cols

[Document(page_content='Country code 3-character', metadata={'year': 2022, 'original_col_name': 'CNT'}),
 Document(page_content='Intl. School ID', metadata={'year': 2022, 'original_col_name': 'CNTSCHID'}),
 Document(page_content='PISA Assessment Cycle (2 digits + 2 character Assessment type - MS/FT)', metadata={'year': 2022, 'original_col_name': 'CYC'}),
 Document(page_content='National Centre 6-digit Code', metadata={'year': 2022, 'original_col_name': 'NatCen'}),
 Document(page_content='Stratum ID 5-character (cnt + original stratum ID)', metadata={'year': 2022, 'original_col_name': 'STRATUM'}),
 Document(page_content='Adjudicated sub-region code 7-digit code (3-digit country code + region ID + stratum ID)', metadata={'year': 2022, 'original_col_name': 'SUBNATIO'}),
 Document(page_content='REGION', metadata={'year': 2022, 'original_col_name': 'REGION'}),
 Document(page_content='OECD country', metadata={'year': 2022, 'original_col_name': 'OECD'}),
 Document(page_content='Mode of Respon

In [7]:
cols_retriever = cols_vectorstore.as_retriever()

In [8]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(temperature=0)

In [9]:
def extract_column_name_hints(question):
    prompt1 = f"Please list the typical database column fields, that required to answer the following question: {question}"
    relevant_col_list_msg = llm.invoke(prompt1)
    relevant_col_list = relevant_col_list_msg.content
    return relevant_col_list

In [10]:
from langchain_community.document_transformers import EmbeddingsRedundantFilter

def match_column_names(hints_text, retriever):
    filter = EmbeddingsRedundantFilter(embeddings=OpenAIEmbeddings())
    res = []
    hints = hints_text.split('\n')
    for hint in hints:
        rel_col_docs = cols_retriever.invoke(hint)    
        rel_col_docs = filter.transform_documents(rel_col_docs)
        res = res+rel_col_docs
    return res

def docs2explanation(docs, meta):
    t = ''
    for idx, doc in enumerate(docs):
        col_name = doc.metadata['original_col_name']
        if meta.variable_measure[col_name] != 'unknown':
            scale = meta.variable_measure[col_name]            
            if scale == 'scale':
                scale = 'interval'                
            measure = ' A ' + meta.readstat_variable_types[col_name] + ' variable with ' +scale  + ' scale measure.'
        else:
            measure = ''
        t =  t + str(idx+1)+ '. ' + col_name + " : " + doc.page_content +'.' + measure +'\n'
    t = t + ''
    return t

In [11]:
def gen_code(question, rel_col_docs, meta_sch):
    data_explanation = docs2explanation(rel_col_docs, meta_sch)
    columns = [i.metadata['original_col_name'] for i in rel_col_docs]
    prompt2 = f"Given a dataframe with the following columns {columns}, column meaning: {data_explanation}, can you generate a python code, without sample data, which can answer the following question? the code must contain only one function called 'run', that returns an exact number. \nQuestion: {question}"
    res = llm.invoke(prompt2)
    code = res.content.replace('```python','').replace('```','')
    return code

def exec_code(code, df):    
    loc = {}
    exec(code + "\nr = run(df)\n", {'df':df}, loc)
    return loc['r']


In [12]:
def pipeline(question, df, meta, col_retriever):
    col_hints = extract_column_name_hints(question)
    print(col_hints)
    rel_col_docs = match_column_names(col_hints, col_retriever)    
    print([i.page_content for i in rel_col_docs])
    code =  gen_code(question, rel_col_docs, meta_sch)    
    print(code)
    res = exec_code(code, df)    
    return res

In [12]:
pipeline('in how many languages ​​was the questionnaire filled out?', df_sch, meta_sch, cols_retriever)

1. Respondent ID
2. Language field
3. Date and time of questionnaire completion
['Mode of Respondent', 'Intl. School ID', 'Language of Questionnaire', 'Stratum ID 5-character (cnt + original stratum ID)', 'Language of Questionnaire', 'Class size (test language class)', 'How many foreign languages are being taught at your school <this academic year>?', 'Percentage [15-year-old modal grade] students who: Students whose [heritage language] is different from [test language]', 'Language of Questionnaire', 'Date of the database creation', 'Mode of Respondent', '<This academic year>,follow. activities/school offers<national modal grade for 15-year-olds>? <country specific item>']

import pandas as pd

def run(df):
    num_languages = df['LANGTEST_QQQ'].nunique()
    return num_languages



54

In [13]:
pipeline('What is Hungary average teacher-student ratio?', df_sch, meta_sch, cols_retriever)

1. Country
2. School
3. Number of teachers
4. Number of students
['REGION', 'OECD country', 'Senate Weight (sum of 5000 per country)', 'Country code 3-character', 'School type', 'School size (Sum)', 'School selectivity', 'Is your school a public or a private school?', 'Proportion of mathematics teachers at school', 'Student-teacher ratio', 'Total number of non-teaching staff at school', 'Student-mathematics teacher ratio', 'Student-teacher ratio', 'School size (Sum)', 'Student-mathematics teacher ratio', 'Total number of mathematics teachers at school (Sum)']

import pandas as pd

def run(data):
    hungary_data = data[data['CNT'] == 'HUN']
    avg_teacher_student_ratio = hungary_data['STRATIO'].mean()
    
    return avg_teacher_student_ratio



10.829131944444445

In [14]:
pipeline('What is OECD average teacher-student ratio?', df_sch, meta_sch, cols_retriever)

1. Country
2. Teacher-student ratio
3. OECD average
['REGION', 'OECD country', 'Senate Weight (sum of 5000 per country)', 'Country code 3-character', 'Student-teacher ratio', 'Student-mathematics teacher ratio', 'Proportion of mathematics teachers at school', 'Proportion of school management personnel', 'OECD country', 'Proportion of all teachers with ISCED level 8 Doctoral qualification', 'Proportion of all teachers with at least ISCED level 6 Bachelor qualification']

import pandas as pd

def run(data):
    oecd_avg_stratio = data[data['OECD'] == 1]['STRATIO'].mean()
    return oecd_avg_stratio



12.551608245736977

In [15]:
pipeline('Where does Hungary stand compared to the OECD average in terms of teacher-student ratio? Give the difference in number.', df_sch, meta_sch, cols_retriever)

1. Country (e.g. Hungary)
2. Teacher-student ratio in Hungary
3. OECD average teacher-student ratio
['OECD country', 'REGION', 'Senate Weight (sum of 5000 per country)', 'Country code 3-character', 'Student-teacher ratio', 'Student-mathematics teacher ratio', 'Proportion of mathematics teachers at school', 'Proportion of school management personnel', 'Student-teacher ratio', 'Student-mathematics teacher ratio', 'Proportion of mathematics teachers at school', 'Proportion of all teachers with at least ISCED level 7 Master qualification']

import pandas as pd

def run(data):
    oecd_avg_stratio = data[data['OECD'] == 1]['STRATIO'].mean()
    hungary_stratio = data[data['CNT'] == 'HUN']['STRATIO'].values[0]
    
    difference = hungary_stratio - oecd_avg_stratio
    
    return difference



2.883191754263022

In [16]:
pipeline('Where does Hungary stand compared to the OECD average in terms of average teacher-student ratio? Give the difference in number.', df_sch, meta_sch, cols_retriever)

1. Country (e.g. Hungary)
2. Average teacher-student ratio in Hungary
3. OECD average teacher-student ratio
4. Difference in number between Hungary and OECD average
['OECD country', 'REGION', 'Senate Weight (sum of 5000 per country)', 'Country code 3-character', 'Student-teacher ratio', 'Student-mathematics teacher ratio', 'Proportion of mathematics teachers at school', 'Proportion of school management personnel', 'Student-teacher ratio', 'Student-mathematics teacher ratio', 'Proportion of mathematics teachers at school', 'Proportion of all teachers with at least ISCED level 7 Master qualification', 'OECD country', 'Proportion of mathematics teachers at school', 'Proportion of all teachers with ISCED level 8 Doctoral qualification', 'Student-mathematics teacher ratio']

import pandas as pd

def run(data):
    # Filter data for Hungary
    hungary_data = data[data['CNT'] == 'HUN']
    
    # Calculate average teacher-student ratio for Hungary
    hungary_avg_ratio = (hungary_data['STRAT

45.16579175426302

In [15]:
pipeline('Where is average teacher-student ratio in Hungary compared to the OECD average in terms of average teacher-student ratio? Give the difference as number.', df_sch, meta_sch, cols_retriever)

1. Country (e.g. Hungary)
2. Average teacher-student ratio in Hungary
3. OECD average teacher-student ratio
['OECD country', 'REGION', 'Senate Weight (sum of 5000 per country)', 'Country code 3-character', 'Student-teacher ratio', 'Student-mathematics teacher ratio', 'Proportion of mathematics teachers at school', 'Proportion of school management personnel', 'Student-teacher ratio', 'Student-mathematics teacher ratio', 'Proportion of mathematics teachers at school', 'Proportion of all teachers with at least ISCED level 7 Master qualification']
def run(df):
    hungary_avg_ratio = df[df['CNT'] == 'HUN']['STRATIO'].mean()
    oecd_avg_ratio = df[df['OECD'] == 1]['STRATIO'].mean()
    
    difference = hungary_avg_ratio - oecd_avg_ratio
    
    return difference


-1.7224763012925326

In [14]:
pipeline('What is average teacher-student ratio in Hungary compared to the OECD average ? Give the difference in number.', df_sch, meta_sch, cols_retriever)

1. Country (e.g. Hungary, OECD)
2. Teacher count
3. Student count
4. Average teacher-student ratio
['OECD country', 'REGION', 'How often students assessed by: Mandatory [standardised tests], e.g. [country specific example]', 'Student-teacher ratio', 'Total number of mathematics teachers at school (Sum)', 'Student-mathematics teacher ratio', 'Student-teacher ratio', 'School size (Sum)', 'Student-mathematics teacher ratio', 'Total number of all teachers at school (Sum)', 'Student-teacher ratio', 'Student-mathematics teacher ratio', 'Proportion of mathematics teachers at school', 'Proportion of school management personnel']

import pandas as pd

def run(data):
    hungary_avg_ratio = data[data['OECD'] == 'Hungary']['STRATIO'].mean()
    oecd_avg_ratio = data[data['OECD'] == 'OECD']['STRATIO'].mean()
    
    difference = hungary_avg_ratio - oecd_avg_ratio
    
    return difference



nan

In [26]:
pipeline('Which country has the highest class sizes? How much is it? ', df_sch, meta_sch, cols_retriever)

1. Country
2. Class size
['REGION', 'OECD country', 'Senate Weight (sum of 5000 per country)', 'Country code 3-character', 'Math class size', 'Class size (test language class)', 'School size (Sum)', 'Student-teacher ratio']

import pandas as pd

def run(data):
    max_class_size = data['CLSIZE'].max()
    country_with_max_class_size = data.loc[data['CLSIZE'] == max_class_size, 'CNT'].values[0]
    
    return max_class_size, country_with_max_class_size



(53.0, 'ALB')

In [27]:
pipeline('Which country has the highest average class sizes? How much is it? ', df_sch, meta_sch, cols_retriever)

1. Country
2. Class size
['REGION', 'OECD country', 'Senate Weight (sum of 5000 per country)', 'Country code 3-character', 'Math class size', 'Class size (test language class)', 'School size (Sum)', 'Student-teacher ratio']

import pandas as pd

def run(data):
    avg_class_sizes = data.groupby('CNT')['CLSIZE'].mean()
    country_highest_avg_class_size = avg_class_sizes.idxmax()
    highest_avg_class_size = avg_class_sizes.max()
    
    return highest_avg_class_size

# Example usage
# data = pd.read_csv('data.csv')
# result = run(data)
# print(result)



42.65

In [28]:
pipeline('Which country has the highest average  teacher student ratio? How much is it? ', df_sch, meta_sch, cols_retriever)

1. Country
2. Teacher student ratio
['REGION', 'OECD country', 'Senate Weight (sum of 5000 per country)', 'Country code 3-character', 'Student-teacher ratio', 'Student-mathematics teacher ratio', 'Proportion of mathematics teachers at school', 'Proportion of school management personnel']

import pandas as pd

def run(data):
    highest_avg_ratio = data.groupby('CNT')['STRATIO'].mean().max()
    country_highest_avg_ratio = data.groupby('CNT')['STRATIO'].mean().idxmax()
    
    return highest_avg_ratio, country_highest_avg_ratio



(28.952087700534758, 'PHL')

In [29]:
pipeline('how many education systems were involved in the survey? ', df_sch, meta_sch, cols_retriever)

1. Education System Name
2. Education System ID
3. Location
4. Number of Schools
5. Number of Students
6. Number of Teachers
7. Number of Administrators
8. Funding Source
9. Curriculum Type
10. Language of Instruction
['School type', "Exist at your school: Written specification of the school's curricular profile and educational goals", 'School size (Sum)', "Extent structures your school's math programme: Discretion of individual teachers", 'Intl. School ID', 'School type', "Exist at your school: Written specification of the school's curricular profile and educational goals", 'School size (Sum)', 'REGION', 'During admission, how often consider: Residence in a particular area', 'Which of the following statements best describes the schooling available to students in your location?', 'Which of the following definitions best describes the community in which your school is located?', 'School size (Sum)', 'Total number of all teachers at school (Sum)', 'Proportion of school management personn

21629

In [30]:
pipeline('how many countries were involved in the survey? ', df_sch, meta_sch, cols_retriever)

1. Country Name
2. Country Code
['Country code 3-character', 'REGION', 'OECD country', 'National Centre 6-digit Code', 'Country code 3-character', 'National Centre 6-digit Code', 'Adjudicated sub-region code 7-digit code (3-digit country code + region ID + stratum ID)', 'REGION']
def run(df):
    num_countries = df['CNT'].nunique()
    return num_countries


80

In [31]:
pipeline('how many OECD countries were involved in the survey? ', df_sch, meta_sch, cols_retriever)

1. Country Name
2. OECD Membership Status
['Country code 3-character', 'REGION', 'OECD country', 'National Centre 6-digit Code', 'OECD country', "During admission, how often consider: Student's working status", 'School selectivity', 'Proportion of all teachers with ISCED level 8 Doctoral qualification']

import pandas as pd

def run(data):
    num_oecd_countries = data['OECD'].nunique()
    return num_oecd_countries



2

In [32]:
pipeline('how many countries that are OECD memeber were involved in the survey? ', df_sch, meta_sch, cols_retriever)

1. Country Name
2. OECD Membership Status
3. Survey Participation Status
['Country code 3-character', 'REGION', 'OECD country', 'National Centre 6-digit Code', 'OECD country', "During admission, how often consider: Student's working status", 'School selectivity', 'Proportion of all teachers with ISCED level 8 Doctoral qualification', "During admission, how often consider: Student's working status", 'Mode of Respondent', 'Proportion parent/guardians who: Participated in local school government (e.g. parent council or school management committee)', 'Language of Questionnaire']

import pandas as pd

def run(data):
    oecd_countries = data[data['OECD'] == 1]['CNT'].nunique()
    return oecd_countries



37

In [33]:
pipeline('how many schools were involved in the survey? ', df_sch, meta_sch, cols_retriever)

1. School ID
2. School Name
3. Location
4. Number of Students
5. Number of Teachers
6. Survey Participation Status
['Intl. School ID', 'School type', "Exist at your school: Written specification of the school's curricular profile and educational goals", 'School size (Sum)', 'School type', 'School size (Sum)', 'Is your school a public or a private school?', 'Intl. School ID', 'REGION', 'During admission, how often consider: Residence in a particular area', 'Which of the following statements best describes the schooling available to students in your location?', 'Which of the following definitions best describes the community in which your school is located?', 'Student-teacher ratio', 'School size (Sum)', 'Student-mathematics teacher ratio', 'Total number of mathematics teachers at school (Sum)', 'Proportion of mathematics teachers at school', 'Total number of all teachers at school (Sum)', 'Total number of non-teaching staff at school', 'Student-teacher ratio', "During admission, how oft

21629

In [35]:

pipeline('How many schools are the answers from?', df_sch, meta_sch, cols_retriever)

1. School ID
2. School Name
3. School Location
4. School Type
5. School District
6. School Contact Information
7. School Enrollment
8. School Website
9. School Principal
10. School Grade Levels
['Intl. School ID', 'School type', "Exist at your school: Written specification of the school's curricular profile and educational goals", 'School size (Sum)', 'School type', 'School size (Sum)', 'Is your school a public or a private school?', 'Intl. School ID', 'Which of the following statements best describes the schooling available to students in your location?', 'School type', 'Which of the following definitions best describes the community in which your school is located?', 'School size (Sum)', 'School type', 'Which of the following statements best describes the schooling available to students in your location?', 'School size (Sum)', 'Is your school a public or a private school?', 'School type', 'Which of the following statements best describes the schooling available to students in your lo

21629

In [42]:
df_sch[df_sch['CNTRYID'] != 'HUN']

,CNT,CNTRYID,CNTSCHID,CYC,NatCen,STRATUM,SUBNATIO,REGION,OECD,ADMINMODE,...,SCSUPRT,PROBSCRI,SCPREPBP,SCPREPAP,DIGPREP,W_SCHGRNRABWT,W_FSTUWT_SCH_SUM,W_FSTUWT_SCH_N,SENWT,VER_DAT
0,ALB,8.0,800001.0,08MS,000800,ALB05,0080000,800.0,0.0,2.0,...,2.0,0.7965,-0.8314,0.8462,0.5908,1.43376,160.58080,41.0,5.76182,03MAY23:10:11:34
1,ALB,8.0,800002.0,08MS,000800,ALB07,0080000,800.0,0.0,2.0,...,1.0,-0.5687,-0.8314,0.8462,-0.3475,2.85278,133.71140,36.0,11.46442,03MAY23:10:11:34
2,ALB,8.0,800003.0,08MS,000800,ALB06,0080000,800.0,0.0,2.0,...,1.0,0.1896,-0.8314,-0.8711,0.4409,7.40007,25.61079,3.0,29.73857,03MAY23:10:11:34
3,ALB,8.0,800004.0,08MS,000800,ALB05,0080000,800.0,0.0,2.0,...,NaN,NaN,NaN,NaN,-0.7414,4.34319,192.24130,39.0,17.45392,03MAY23:10:11:34
4,ALB,8.0,800005.0,08MS,000800,ALB03,0080000,800.0,0.0,2.0,...,2.0,0.6649,-0.8314,0.8462,-0.0102,7.52060,45.69535,5.0,30.22293,03MAY23:10:11:34
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21624,QUR,901.0,90100178.0,08MS,080400,UKR26,9010000,90100.0,0.0,2.0,...,NaN,NaN,NaN,NaN,2.2897,53.75000,459.78720,8.0,40.82246,01MAY23:18:14:40
21625,QUR,901.0,90100179.0,08MS,080400,UKR26,9010000,90100.0,0.0,2.0,...,1.0,0.4590,-0.8314,0.8462,-0.2793,53.75000,280.33170,5.0,40.82246,01MAY23:18:14:40
21626,QUR,901.0,90100180.0,08MS,080400,UKR12,9010000,90100.0,0.0,2.0,...,1.0,0.9920,1.3112,0.8462,0.3470,25.49544,1726.41200,45.0,19.36347,01MAY23:18:14:40
21627,QUR,901.0,90100181.0,08MS,080400,UKR21,9010000,90100.0,0.0,2.0,...,1.0,0.1896,0.7399,0.8462,2.2897,18.33558,1684.65500,48.0,13.92564,01MAY23:18:14:40
